In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [ ]:
df=pd.read_csv("/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv")
df.head()

In [ ]:
df["relevant_data"]=df["company_profile"]+df["description"]+df["requirements"]
df=df[["relevant_data","fraudulent"]]
df.dropna(inplace=True)

In [ ]:
from nltk.tokenize import word_tokenize

df.iloc[0,0]

In [ ]:
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

x_data=[]
for i in range(12631):
    x_data.append([word.lower() for word in word_tokenize(df.iloc[i,0]) if not word in stopwords.words('english') and not 'http' in word])


In [ ]:
for i in x_data:
    for x in i:
        x=x[0]

In [ ]:
x_data[0]

In [ ]:
import gensim

w2v_model = gensim.models.Word2Vec(sentences=x_data, size=100, window=2, min_count=0)


In [ ]:
w2v_model.wv.most_similar("talented")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_data)

X = tokenizer.texts_to_sequences(x_data)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
X = pad_sequences(X, maxlen=700)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

embedding_vectors = get_weight_matrix(w2v_model, tokenizer.word_index)

In [ ]:
w2v_model["we"]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

#Non-trainable embeddidng layer
model = Sequential()
model.add(Embedding(vocab_size, output_dim=100, weights=[embedding_vectors], input_length=700, trainable=False))
#LSTM 
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

X_train, X_test, y_train, y_test = train_test_split(X, df["fraudulent"].values) 


In [ ]:
model.fit(X_train, y_train, epochs=12,validation_data=(X_test,y_test))